In [ ]:
import bayesianchangepoint as bcp
import matplotlib.pyplot as plt
import numpy as np
import glob
import pickle
import os
%matplotlib inline
fig_width = 15

In [ ]:
liste = []
for fname in glob.glob('data/*pkl'):
    a = fname.split('/')[1].split('.')[0].split('_')
    liste.append(a)
    #print(a)

In [ ]:
exp = []
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']:
        a = 'data/%s_%s_%s_%s.pkl'%(liste[x][0], liste[x][1],liste[x][2],liste[x][3])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            exp.append(b)

param = []
for x in range(len(liste)) :
    if liste[x][0]=='pari' and liste[x][1] in ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']:
        a = 'parametre/param_Fit_%s.pkl'%(liste[x][1])
        with open(a, 'rb') as fichier :
            b = pickle.load(fichier, encoding='latin1')
            param.append(b)

In [ ]:
liste_p_hat_e_tau = pickle.load(open('liste_p_hat_e_tau.pkl', 'rb' ))
liste_p_hat_m_tau = pickle.load(open('liste_p_hat_m_tau.pkl', 'rb' ))

In [ ]:
from scipy import stats
def regress(p, b) :
    p=np.asarray(p)
    b=np.asarray(b)
    slope, intercept, r_, p_value, std_err = stats.linregress(p[~np.isnan(b)], b[~np.isnan(b)])
    return r_

tau max 500?

In [ ]:
tau_max = 200
TAU = np.arange(0,tau_max,1)

fig_width=10
t_titre=25
t_label=20

# Test par sujet

In [ ]:
liste_tau = {}
liste_r = {}

liste_p_hat_tau = liste_p_hat_e_tau.copy()

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    liste_tau[s] = {}
    liste_r[s] = {}
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    
    liste_tau[s] = {}
    liste_r[s] = {}
    
    r_results, r_va = [], []
    for t in range(tau_max) :
        tau = TAU[t]
        phat = []
        res, va = [], []
        for block in range(N_blocks) :
            phat.extend(liste_p_hat_tau['%s'%tau][block])
            res.extend(results[:, block])
            va.extend(v_anti[block])
            
        r_results.append(regress(phat, res))
        r_va.append(regress(phat, va))
        
    liste_tau[s]['result'] = [r_results.index(np.nanmax(r_results)), np.nanmax(r_results)]
    liste_tau[s]['va'] = [r_va.index(np.nanmax(r_va)), np.nanmax(r_va)]

    liste_r[s]['result'] = r_results
    liste_r[s]['va'] = r_va
        
#-------------------------------------------------------------------------------------------
fig, axs = plt.subplots(len(exp), 1, figsize=(1*fig_width, len(exp)*fig_width/(1.6180)))

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet

        
    axs[x].plot(TAU[:tau_max], liste_r[s]['result'], c='k', label='result')
    axs[x].plot(TAU[:tau_max], liste_r[s]['va'], c='r', label='va')
    axs[x].axis([-12.8,tau_max+12.8,-0.016,.516])

    axs[x].scatter(liste_tau[s]['result'][0], liste_tau[s]['result'][1], color='K')
    axs[x].scatter(liste_tau[s]['va'][0], liste_tau[s]['va'][1], color='r')

    axs[x].set_title('sujet : %s'%(s), fontsize=t_label)

    axs[x].set_ylabel('Correlation', fontsize=t_label)
    axs[x].set_yticks(np.arange(0,1.25,0.25))

    axs[x].set_xticks(np.arange(0,tau_max+1,int(tau_max/4)))

    if x == len(exp)-1 :
        axs[x].set_xlabel('tau', fontsize=t_label)
        #axs[x].set_xticks(np.arange(0,501,100))
    #else :
    #    axs[x].set_xticks([])
    axs[x].tick_params(direction='out', right=False, top=False,  labelsize=15)
    if x == 0 :
        axs[x].legend(ncol=1, mode='expand', frameon=False, bbox_to_anchor=(1, 0, 1, 1), fontsize=t_label/1.2, markerscale=100)

plt.show()
#-------------------------------------------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=((fig_width*2), (fig_width)/(1.6180)))
plt.suptitle('par sujet', fontsize=20, y=1.05)

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    axs[0].scatter(x, liste_tau[s]['result'][0], color='K')
    axs[1].scatter(x, liste_tau[s]['va'][0], color='r')
        

axs[0].set_title('result', fontsize=20)
axs[0].set_ylabel('tau$_{result}$', fontsize=14)
axs[0].axis([-1, 12, 0, 500])

axs[1].set_title('VA', fontsize=20)
axs[1].set_ylabel('tau$_{va}$', fontsize=14)
axs[1].axis([-1, 12, 0, 500])

plt.show()
#-------------------------------------------------------------------------------------------
colors = ['black','dimgrey','grey','darkgrey','silver','rosybrown','lightcoral','indianred','firebrick','brown','darkred','red']

fig, axs = plt.subplots(1, 1, figsize=(1*fig_width, 1*fig_width/(1.6180)))
#plt.subplots_adjust(hspace=0.05, wspace=0.05/(1.6180))


for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    axs.scatter(liste_tau[s]['result'][0], liste_tau[s]['va'][0], color=colors[x])

axs.set_xlabel('Tau Results', fontsize=t_label)
axs.set_ylabel('Tau VA', fontsize=t_label)
axs.tick_params(direction='out', right=False, top=False,  labelsize=15)
axs.axis([0,tau_max,0,tau_max])
plt.show()
#-------------------------------------------------------------------------------------------

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(fig_width*2, (fig_width*1)/1.6180))
plt.subplots_adjust(hspace=0.4)

p_r = []
p_va = []

res = []
va = []

for x in range(len(exp)) :
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    p = exp[x]['p']
    s = exp[x]['observer'] # sujet
    tau_r, tau_va =  liste_tau[s]['result'][0], liste_tau[s]['va'][0]
    h_r, h_va = 1./tau_r, 1./tau_va 
    
    for block in range(N_blocks):
        p_hat_r, p_hat_va = [], []
        #----------------------------------------------------
        # Pour chaque pause !
        #----------------------------------------------------
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_r, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_r.extend(p_hat)
            p_r.extend(p_hat)

            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_va, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_va.extend(p_hat)
            p_va.extend(p_hat)
        
        
        res.extend(results[:, block])
        va.extend(v_anti[block])
        axs[0].scatter(p_hat_r, results[:, block], c=colors[x], alpha=0.5, linewidths=0)
        axs[1].scatter(p_hat_va, v_anti[block], c=colors[x], alpha=0.5, linewidths=0)

        
p_r = np.ma.masked_array(p_r, mask=np.isnan(res)).compressed()
res = np.ma.masked_array(res, mask=np.isnan(res)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_r, res)
x_test = np.linspace(np.min(p_r), np.max(p_r), 100)
fitLine = slope * x_test + intercept
y_1 = -0.032
y_2 = 1.032

axs[0].plot(x_test, fitLine, c='k', linewidth=2)
axs[0].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
        
p_va = np.ma.masked_array(p_va, mask=np.isnan(va)).compressed()
va = np.ma.masked_array(va, mask=np.isnan(va)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_va, va)
x_test = np.linspace(np.min(p_va), np.max(p_va), 100)
fitLine = slope * x_test + intercept
y_1 = -21.28
y_2 = 21.28
axs[1].plot(x_test, fitLine, c='k', linewidth=2)
axs[1].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
              
plt.show()
    

# Test avec les blocs

In [ ]:
liste_tau = {}
liste_r = {}

liste_p_hat_tau = liste_p_hat_e_tau.copy()

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    liste_tau[s] = {}
    liste_r[s] = {}
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    
    for block in range(N_blocks) :

        liste_tau[s][block] = {}
        liste_r[s][block] = {}
        
        
        r_results, r_va = [], []
        for t in range(tau_max) :
            tau = TAU[t]
            phat = liste_p_hat_tau['%s'%tau][block]
            
            r_results.append(regress(phat, results[:, block]))
            r_va.append(regress(phat, v_anti[block]))
        
        liste_tau[s][block]['result'] = [r_results.index(np.nanmax(r_results)), np.nanmax(r_results)]
        liste_tau[s][block]['va'] = [r_va.index(np.nanmax(r_va)), np.nanmax(r_va)]
        
        liste_r[s][block]['result'] = r_results
        liste_r[s][block]['va'] = r_va
#-------------------------------------------------------------------------------------------
marker = [ '-','--','-.']

fig, axs = plt.subplots(len(exp), 1, figsize=(1*fig_width, len(exp)*fig_width/(1.6180)))
#plt.subplots_adjust(hspace=0.05, wspace=0.05/(1.6180))


for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        
        axs[x].plot(TAU[:tau_max], liste_r[s][block]['result'], c='k', ls=marker[block], label='block %s result'%(block))
        axs[x].plot(TAU[:tau_max], liste_r[s][block]['va'], c='r', ls=marker[block], label='block %s va'%(block))
        axs[x].axis([-12.8,tau_max+12.8,-0.016,.516])

        axs[x].scatter(liste_tau[s][block]['result'][0], liste_tau[s][block]['result'][1], color='K')
        axs[x].scatter(liste_tau[s][block]['va'][0], liste_tau[s][block]['va'][1], color='r')
        
        axs[x].set_title('sujet : %s'%(s), fontsize=t_label)

        axs[x].set_ylabel('Correlation', fontsize=t_label)
        axs[x].set_yticks(np.arange(0,1.25,0.25))

        axs[x].set_xticks(np.arange(0,tau_max+1,int(tau_max/4)))

        if x == len(exp)-1 :
            axs[x].set_xlabel('tau', fontsize=t_label)
            #axs[x].set_xticks(np.arange(0,501,100))
        #else :
        #    axs[x].set_xticks([])
        axs[x].tick_params(direction='out', right=False, top=False,  labelsize=15)
        if x == 0 :
            axs[x].legend(ncol=1, mode='expand', frameon=False, bbox_to_anchor=(1, 0, 1, 1), fontsize=t_label/1.2, markerscale=100)
plt.show()
#-------------------------------------------------------------------------------------------
marker = ['o', 'v', '^'] 

fig, axs = plt.subplots(1, 2, figsize=((2*fig_width), (fig_width)/(1.6180)))
plt.suptitle('par sujet', fontsize=20, y=1.05)

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        axs[0].scatter(x, liste_tau[s][block]['result'][0], color='K', marker=marker[block])
        axs[1].scatter(x, liste_tau[s][block]['va'][0], color='r', marker=marker[block])
        

axs[0].set_title('result', fontsize=20)
axs[0].set_ylabel('tau$_{result}$', fontsize=14)
#axs[0].axis([-1, 12, -0.03, 0.28])

axs[1].set_title('VA', fontsize=20)
axs[1].set_ylabel('tau$_{va}$', fontsize=14)
#axs[1].axis([-1, 12, -0.03, 0.28])

plt.show()
#-------------------------------------------------------------------------------------------
colors = ['black','dimgrey','grey','darkgrey','silver','rosybrown','lightcoral','indianred','firebrick','brown','darkred','red']
alpha = [.6,.75,1]
marker = ['o', 'v', '^'] 

fig, axs = plt.subplots(1, 1, figsize=(1*fig_width, 1*fig_width/(1.6180)))
#plt.subplots_adjust(hspace=0.05, wspace=0.05/(1.6180))


for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        axs.scatter(liste_tau[s][block]['result'][0], liste_tau[s][block]['va'][0], color=colors[x], marker=marker[block])#, alpha=alpha[block], label='%s %s'%(s, block))

axs.set_xlabel('Tau Results', fontsize=t_label)
axs.set_ylabel('Tau VA', fontsize=t_label)
axs.tick_params(direction='out', right=False, top=False,  labelsize=15)
#axs.legend(ncol=1, mode='expand', frameon=False, bbox_to_anchor=(1, 0, 1, 1), fontsize=t_label/1.2, markerscale=100)
axs.axis([0,tau_max,0,tau_max])
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(fig_width*2, (fig_width*1)/1.6180))
plt.subplots_adjust(hspace=0.4)

p_r = []
p_va = []

res = []
va = []

for x in range(len(exp)) :
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    p = exp[x]['p']
    s = exp[x]['observer'] # sujet

    
    for block in range(N_blocks):
        tau_r, tau_va =  liste_tau[s][block]['result'][0], liste_tau[s][block]['va'][0]
        h_r, h_va = 1./tau_r, 1./tau_va 
        
        p_hat_r, p_hat_va = [], []
        #----------------------------------------------------
        # Pour chaque pause !
        #----------------------------------------------------
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_r, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_r.extend(p_hat)
            p_r.extend(p_hat)

            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_va, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_va.extend(p_hat)
            p_va.extend(p_hat)
        
        
        res.extend(results[:, block])
        va.extend(v_anti[block])
        axs[0].scatter(p_hat_r, results[:, block], c=colors[x], alpha=0.5, linewidths=0)
        axs[1].scatter(p_hat_va, v_anti[block], c=colors[x], alpha=0.5, linewidths=0)

        
p_r = np.ma.masked_array(p_r, mask=np.isnan(res)).compressed()
res = np.ma.masked_array(res, mask=np.isnan(res)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_r, res)
x_test = np.linspace(np.min(p_r), np.max(p_r), 100)
fitLine = slope * x_test + intercept
y_1 = -0.032
y_2 = 1.032

axs[0].plot(x_test, fitLine, c='k', linewidth=2)
axs[0].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
        
p_va = np.ma.masked_array(p_va, mask=np.isnan(va)).compressed()
va = np.ma.masked_array(va, mask=np.isnan(va)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_va, va)
x_test = np.linspace(np.min(p_va), np.max(p_va), 100)
fitLine = slope * x_test + intercept
y_1 = -21.28
y_2 = 21.28
axs[1].plot(x_test, fitLine, c='k', linewidth=2)
axs[1].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
              
plt.show()
    

# pour chaque pause

In [ ]:
liste_tau = {}
liste_r = {}

liste_p_hat_tau = liste_p_hat_e_tau.copy()

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    liste_tau[s] = {}
    liste_r[s] = {}
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    
    for block in range(N_blocks) :
        liste_tau[s][block] = {}
        liste_r[s][block] = {}
        #----------------------------------------------------
        # Pour chaque pause !
        #----------------------------------------------------
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            liste_tau[s][block][a] = {}
            liste_r[s][block][a] = {}
            
            r_results, r_va = [], []
            for t in range(tau_max) :
                tau = TAU[t]
                phat = liste_p_hat_tau['%s'%tau][block][liste[a]:liste[a+1]]
                
                r_results.append(regress(phat, results[liste[a]:liste[a+1], block]))
                r_va.append(regress(phat, v_anti[block][liste[a]:liste[a+1]]))
            
            liste_tau[s][block][a]['result'] = [r_results.index(np.nanmax(r_results)), np.nanmax(r_results)]
            liste_tau[s][block][a]['va'] = [r_va.index(np.nanmax(r_va)), np.nanmax(r_va)]

            liste_r[s][block][a]['result'] = r_results
            liste_r[s][block][a]['va'] = r_va
#-------------------------------------------------------------------------------------------
marker = [ '-','--','-.']
alpha = [0.4, 0.6,0.8,1]
fig, axs = plt.subplots(len(exp), 1, figsize=(1*fig_width, len(exp)*fig_width/(1.6180)))
#plt.subplots_adjust(hspace=0.05, wspace=0.05/(1.6180))


for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            axs[x].plot(TAU[:tau_max], liste_r[s][block][a]['result'], c='k', ls=marker[block], alpha=alpha[a], label='block %s %s result'%(block, a))
            axs[x].plot(TAU[:tau_max], liste_r[s][block][a]['va'], c='r', ls=marker[block], alpha=alpha[a], label='block %s %s va'%(block, a))
            axs[x].axis([-12.8,tau_max+12.8,-0.016,.516])

            axs[x].scatter(liste_tau[s][block][a]['result'][0], liste_tau[s][block][a]['result'][1], color='K')
            axs[x].scatter(liste_tau[s][block][a]['va'][0], liste_tau[s][block][a]['va'][1], color='r')

            axs[x].set_title('%s'%(s), fontsize=t_label)

            axs[x].set_ylabel('Correlation', fontsize=t_label)
            axs[x].set_yticks(np.arange(0,1.25,0.25))

            axs[x].set_xticks(np.arange(0,tau_max+1,int(tau_max/4)))

        if x == len(exp)-1 :
            axs[x].set_xlabel('tau', fontsize=t_label)
            #axs[x].set_xticks(np.arange(0,501,100))
        #else :
        #    axs[x].set_xticks([])
        axs[x].tick_params(direction='out', right=False, top=False,  labelsize=15)
        if x == 0 :
            axs[x].legend(ncol=1, mode='expand', frameon=False, bbox_to_anchor=(1, 0, 1, 1), fontsize=t_label/1.2, markerscale=100)
plt.show()
#-------------------------------------------------------------------------------------------
marker = ['o', 'v', '^'] 

fig, axs = plt.subplots(1, 2, figsize=((2*fig_width), (fig_width)/(1.6180)))
plt.suptitle('par sujet', fontsize=20, y=1.05)

for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            axs[0].scatter(x, liste_tau[s][block][a]['result'][0], color='K', marker=marker[block], alpha=alpha[a])
            axs[1].scatter(x, liste_tau[s][block][a]['va'][0], color='r', marker=marker[block], alpha=alpha[a])


axs[0].set_title('result', fontsize=20)
axs[0].set_ylabel('tau$_{result}$', fontsize=14)
#axs[0].axis([-1, 12, -0.03, 0.28])

axs[1].set_title('VA', fontsize=20)
axs[1].set_ylabel('tau$_{va}$', fontsize=14)
#axs[1].axis([-1, 12, -0.03, 0.28])

plt.show()
#-------------------------------------------------------------------------------------------
colors = ['black','dimgrey','grey','darkgrey','silver','rosybrown','lightcoral','indianred','firebrick','brown','darkred','red']
alpha = [.6,.75,1]

fig, axs = plt.subplots(1, 1, figsize=(1*fig_width, 1*fig_width/(1.6180)))
#plt.subplots_adjust(hspace=0.05, wspace=0.05/(1.6180))


for x in range(len(exp)) :
    s = exp[x]['observer'] # sujet
    for block in range(N_blocks) :
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            axs.scatter(liste_tau[s][block][a]['result'][0], liste_tau[s][block][a]['va'][0], color=colors[x], marker=marker[block])#, alpha=alpha[block], label='%s %s'%(s, block))

axs.set_xlabel('Tau Results', fontsize=t_label)
axs.set_ylabel('Tau VA', fontsize=t_label)
axs.tick_params(direction='out', right=False, top=False,  labelsize=15)
#axs.legend(ncol=1, mode='expand', frameon=False, bbox_to_anchor=(1, 0, 1, 1), fontsize=t_label/1.2, markerscale=100)
axs.axis([0,tau_max,0,tau_max])
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(fig_width*2, (fig_width*1)/1.6180))
plt.subplots_adjust(hspace=0.4)

p_r = []
p_va = []

res = []
va = []

for x in range(len(exp)) :
    
    N_blocks = exp[x]['N_blocks']
    N_trials = exp[x]['N_trials']
    results = (exp[x]['results']+1)/2 # results est sur [-1,1] on le ramene sur [0,1]
    v_anti = param[x]['v_anti']
    p = exp[x]['p']
    s = exp[x]['observer'] # sujet

    for block in range(N_blocks):
        p_hat_r, p_hat_va = [], []
        liste = [0,50,100,150,200]
        for a in range(len(liste)-1) :
            tau_r, tau_va =  liste_tau[s][block][a]['result'][0], liste_tau[s][block][a]['va'][0]
            h_r, h_va = 1./tau_r, 1./tau_va 
            
            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_r, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_r.extend(p_hat)
            p_r.extend(p_hat)

            p_bar, r, beliefs = bcp.inference(p[liste[a]:liste[a+1], block, 0], h=h_va, p0=.5)
            p_hat, r_hat = bcp.readout(p_bar, r, beliefs, mode='expectation')
            p_hat_va.extend(p_hat)
            p_va.extend(p_hat)
        
        
        res.extend(results[:, block])
        va.extend(v_anti[block])
        axs[0].scatter(p_hat_r, results[:, block], c=colors[x], alpha=0.5, linewidths=0)
        axs[1].scatter(p_hat_va, v_anti[block], c=colors[x], alpha=0.5, linewidths=0)

        
p_r = np.ma.masked_array(p_r, mask=np.isnan(res)).compressed()
res = np.ma.masked_array(res, mask=np.isnan(res)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_r, res)
x_test = np.linspace(np.min(p_r), np.max(p_r), 100)
fitLine = slope * x_test + intercept
y_1 = -0.032
y_2 = 1.032

axs[0].plot(x_test, fitLine, c='k', linewidth=2)
axs[0].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
        
p_va = np.ma.masked_array(p_va, mask=np.isnan(va)).compressed()
va = np.ma.masked_array(va, mask=np.isnan(va)).compressed()
slope, intercept, r_value, p_value, std_err = stats.linregress(p_va, va)
x_test = np.linspace(np.min(p_va), np.max(p_va), 100)
fitLine = slope * x_test + intercept
y_1 = -21.28
y_2 = 21.28
axs[1].plot(x_test, fitLine, c='k', linewidth=2)
axs[1].text(0.8,y_1+(y_2-y_1)/10, 'r = %0.3f'%(r_value), fontsize=t_label)
              
plt.show()
    